<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/NLP/RNN_Sequence_Model_IMDB_Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  8952k      0  0:00:09  0:00:09 --:--:-- 16.4M


In [ ]:
!tar -xf aclImdb_v1.tar.gz

In [ ]:
!ls -l

total 82164
drwxr-xr-x 4 7297 1000     4096 Jun 26  2011 aclImdb
-rw-r--r-- 1 root root 84125825 Dec 28 02:31 aclImdb_v1.tar.gz
drwxr-xr-x 1 root root     4096 Dec 20 20:19 sample_data


In [ ]:
!rm -r aclImdb/train/unsup

In [ ]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [ ]:
import os, pathlib, shutil, random

In [ ]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

In [ ]:
for category in ("pos", "neg"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_of_val_files = int(len(files) * 0.2)

  val_files = files[-num_of_val_files:]

  for fname in val_files:
    shutil.move(train_dir / category / fname, val_dir / category / fname)

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
batch_size = 32

In [ ]:
train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size = batch_size)

Found 20000 files belonging to 2 classes.


In [ ]:
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)

Found 5000 files belonging to 2 classes.


In [ ]:
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size = batch_size)

Found 25000 files belonging to 2 classes.


In [ ]:
for inputs, targets in train_ds:
  print(inputs[0])
  print(targets[0])
  print(inputs.shape)
  print(targets.shape)
  print(inputs.dtype)
  print(targets.dtype)
  break

tf.Tensor(b'This was my first, and probably the last Angelopoulos movie. I was eager to get into it, as it featured Mastroianni, one of my favorite actors and was a film By Theo, of whom I\'ve heard a lot. The opening was promising, a long shot over a jeep of soldiers across the Albanian-Greek border. OK! but that was all. Nothing left. The movie had big holes and I don\'t know which to mention first. The main plot of the story is revealed to the journalist by the old woman. during a long walk. It\'s like a 15 minutes monologue, killing the action and viewers patience, nothing happening on screen for 15 or even 20 minutes, apart this old lady telling a story. All that is presumed to be shown through action, was simply told to the camera by the old lady. In a moment, the equippe of TV was heading to the bar. They turn the corner and immediately the winter begins! Probably, shot in different days, continuity leaked. A lot of problems with the story-telling, it went from absurd to irratio

In [ ]:
from keras.layers import TextVectorization

In [126]:
max_length = 600
max_token = 20000

In [127]:
text_vectorization = TextVectorization(output_mode="int", max_tokens=max_token, output_sequence_length=max_length)

In [ ]:
text_ds = train_ds.map(lambda x, y: x)

In [128]:
text_vectorization.adapt(text_ds)

In [129]:
ds = text_vectorization.get_vocabulary()

In [130]:
print(ds)

['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i', 'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'on', 'not', 'you', 'his', 'are', 'have', 'be', 'he', 'one', 'its', 'at', 'all', 'by', 'an', 'they', 'who', 'from', 'so', 'like', 'her', 'just', 'or', 'about', 'has', 'if', 'out', 'some', 'there', 'what', 'good', 'when', 'more', 'very', 'even', 'my', 'she', 'no', 'would', 'up', 'only', 'which', 'really', 'time', 'story', 'were', 'see', 'had', 'their', 'can', 'me', 'we', 'than', 'much', 'well', 'been', 'get', 'will', 'into', 'also', 'bad', 'first', 'people', 'other', 'because', 'do', 'great', 'how', 'most', 'him', 'dont', 'then', 'made', 'make', 'films', 'way', 'movies', 'could', 'them', 'any', 'after', 'too', 'characters', 'think', 'watch', 'many', 'being', 'two', 'seen', 'character', 'never', 'little', 'where', 'acting', 'plot', 'best', 'love', 'did', 'life', 'know', 'show', 'ever', 'does', 'your', 'still', 'better', 'over', 'off', 'end', 'say',

In [131]:
len(ds)

20000

In [132]:
binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [133]:
binary_1gram_val_ds = val_ds.map(lambda x, y : (text_vectorization(x), y), num_parallel_calls=4)

In [134]:
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [ ]:
for inputs, targets in binary_1gram_train_ds:
  print(inputs[0])
  print(targets[0])
  break

tf.Tensor([805.18274    7.673062   9.243181 ...   0.         0.         0.      ], shape=(20000,), dtype=float32)
tf.Tensor(1, shape=(), dtype=int32)


In [ ]:
def get_model(max_tokens=20000, hidden_dims=16):

  inputs = keras.Input(shape=(max_tokens,))
  x = keras.layers.Dense(hidden_dims, activation="relu")(inputs)
  x = keras.layers.Dropout(0.5)(x)
  outputs = keras.layers.Dense(1, activation="sigmoid")(x)

  model = keras.Model(inputs, outputs)
  model.compile(optimizer="rmsprop",
                loss="binary_crossentropy",
                metrics=["accuracy"])
  
  return model


In [ ]:
model = get_model()

In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_6 (Dense)             (None, 16)                320016    
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_tfidf_2gram.keras", save_best_only=True)
]

In [ ]:
history = model.fit(binary_1gram_train_ds.cache(), 
                    validation_data = binary_1gram_val_ds.cache(),
                    callbacks=callbacks,
                    epochs=10)

Epoch 1/10
625/625 [==============================] - 16s 24ms/step - loss: 0.4997 - accuracy: 0.7665 - val_loss: 0.3157 - val_accuracy: 0.8814
Epoch 2/10
625/625 [==============================] - 5s 7ms/step - loss: 0.3621 - accuracy: 0.8533 - val_loss: 0.2980 - val_accuracy: 0.8798
Epoch 3/10
625/625 [==============================] - 5s 7ms/step - loss: 0.3209 - accuracy: 0.8648 - val_loss: 0.2917 - val_accuracy: 0.8916
Epoch 4/10
625/625 [==============================] - 5s 8ms/step - loss: 0.2818 - accuracy: 0.8885 - val_loss: 0.2892 - val_accuracy: 0.8956
Epoch 5/10
625/625 [==============================] - 5s 7ms/step - loss: 0.2636 - accuracy: 0.8982 - val_loss: 0.2970 - val_accuracy: 0.8878
Epoch 6/10
625/625 [==============================] - 5s 7ms/step - loss: 0.2426 - accuracy: 0.9040 - val_loss: 0.2997 - val_accuracy: 0.8920
Epoch 7/10
625/625 [==============================] - 5s 7ms/step - loss: 0.2269 - accuracy: 0.9120 - val_loss: 0.3190 - val_accuracy: 0.8950
Epoc

In [ ]:
#result with 2000 dictionary words
model.evaluate(binary_1gram_test_ds)

782/782 [==============================] - 8s 10ms/step - loss: 0.3984 - accuracy: 0.8443


[0.39842262864112854, 0.8443199992179871]

In [ ]:
#result with 20000 dictionary word
model.evaluate(binary_1gram_test_ds)

782/782 [==============================] - 14s 18ms/step - loss: 0.3729 - accuracy: 0.8802


[0.3729023337364197, 0.8802000284194946]

In [ ]:
#result with 3 ngrams and 20000 dictionary word
model.evaluate(binary_1gram_test_ds)

782/782 [==============================] - 10s 13ms/step - loss: 0.3576 - accuracy: 0.8940


[0.3576336205005646, 0.8939999938011169]

In [ ]:
#result with 2 ngrams, tfidf and 20000 dictionary word
model.evaluate(binary_1gram_test_ds)

782/782 [==============================] - 10s 12ms/step - loss: 0.3531 - accuracy: 0.8847


[0.3531334400177002, 0.8846799731254578]